modelado (paramétrico, no paramétrico y ML) para estimar parámetros de los citricos a partir de variables SAR

**Objetivo (Y)**: estimar parámetros biofísicos de citricos medidos en campo (altura del arbol, diametro de la copa y biomasa).

**Predictores (X)**: derivados de SAR Sentinel‑1

# 1- Carga y preparación de datos

In [1]:
from utils.model_prep import load_dataset
data_path = '../../data/processed/SARCitric.csv'
df = load_dataset(data_path)
# Cambiar el nombre de una columna
df = df.rename(columns={"biomasa_total_m3_kg": "biomass"})
df.head()

,id_point,datetime,tree_height,canopy_diam,biomass,Sigma0_RATIO_VH_VV,Gamma0_RATIO_VH_VV
0,O1-1,2025-02-20 15:23:00,4.16625,4.459995,49.540402,0.810841,0.801533
1,O1-13,2025-02-20 15:36:00,4.41500,4.846434,60.126747,0.810891,0.801586
2,O1-17,2025-02-20 16:06:00,4.16000,4.258532,48.985622,0.810909,0.801612
3,O1-25,2025-02-20 16:15:00,4.86000,4.923015,68.788006,0.810925,0.801633
4,O1-9,2025-02-20 15:30:00,3.25500,4.610760,48.888886,0.810869,0.801555


# 2 - Modeling
## Lineal

In [2]:
from utils.model_prep import select_xy, random_splits

target_col = ['tree_height', 'canopy_diam', 'biomass']
feature_cols = ['Sigma0_RATIO_VH_VV', 'Gamma0_RATIO_VH_VV']
X, y = select_xy(df, target_col, feature_cols)
num_features = list(X.columns)

# Dividir en conjuntos de entrenamiento/validación
splits = random_splits(X, y, val_size=0.3, random_state=42)
(X_train, y_train) = splits["train"]
(X_val,   y_val)   = splits["val"]

In [3]:
out_dir = '../../results/canopy/artifacts_linear'

from models.linear import run_baseline_from_splits
# Ejecutar el modelo lineal para cada variable
for target in target_col:
    print(f"Ejecutando LR para: {target}")
    metrics_LR = run_baseline_from_splits(
        X_train, y_train[target],
        X_val, y_val[target],
        target,
        num_features,
        model_type="ridge",
        alpha=1.0,
        outdir=out_dir
    )
    print(f"Resultados para {target}: {metrics_LR}")

Ejecutando LR para: tree_height
Resultados para tree_height: {'train': {'RMSE': 0.8316349452442439, 'MAE': 0.7163199285737488, 'R2': 0.15539351218728414, 'rRMSE(%)': 30.122800758965994}, 'val': {'RMSE': 0.9249952112902957, 'MAE': 0.7763487672883226, 'R2': 0.08919663580712123, 'rRMSE(%)': 33.15738348831116}}
Ejecutando LR para: canopy_diam
Resultados para canopy_diam: {'train': {'RMSE': 0.7883531938874485, 'MAE': 0.6483857104417027, 'R2': 0.09845064109481627, 'rRMSE(%)': 23.150391718722716}, 'val': {'RMSE': 1.0131042016658034, 'MAE': 0.7749977494864083, 'R2': 0.1621494843231298, 'rRMSE(%)': 29.968426692988814}}
Ejecutando LR para: biomass
Resultados para biomass: {'train': {'RMSE': 16.9159189768231, 'MAE': 13.99742386464722, 'R2': 0.14398351069209858, 'rRMSE(%)': 71.2122214931305}, 'val': {'RMSE': 19.262648248595724, 'MAE': 16.314296846189105, 'R2': 0.13051739816144425, 'rRMSE(%)': 77.23388217643567}}


<Figure size 640x480 with 0 Axes>